In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    DDD_Train_Decile2 = pd.read_pickle("DDD_Train_Decile2.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile2.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile2, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))

    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150,200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 2')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 2s 876us/step - loss: 0.5064 - acc: 0.7769 - val_loss: 0.5061 - val_acc: 0.7690
Epoch 38/100
2264/2264 [==============================] - 2s 820us/step - loss: 0.5025 - acc: 0.7761 - val_loss: 0.5033 - val_acc: 0.7584
Epoch 39/100
2264/2264 [==============================] - 2s 819us/step - loss: 0.4952 - acc: 0.7721 - val_loss: 0.5069 - val_acc: 0.7690
Epoch 40/100
2264/2264 [==============================] - 2s 847us/step - loss: 0.4997 - acc: 0.7765 - val_loss: 0.5036 - val_acc: 0.7619
Epoch 41/100
2264/2264 [==============================] - 2s 859us/step - loss: 0.4971 - acc: 0.7765 - val_loss: 0.5016 - val_acc: 0.7584
Epoch 42/100
2264/2264 [==============================] - 2s 761us/step - loss: 0.4982 - acc: 0.7787 - val_loss: 0.5040 - val_acc: 0.7619
Epoch 43/100
2264/2264 [==============================] - 2s 792us/step - loss: 0.4998 - acc: 0.7756 - val_loss: 0.5041 - val_acc: 0.7672
Epoch 44/100
2264/2264 [=======================

Epoch 96/100
2264/2264 [==============================] - 2s 693us/step - loss: 0.4889 - acc: 0.7800 - val_loss: 0.4992 - val_acc: 0.7584
Epoch 97/100
2264/2264 [==============================] - 2s 766us/step - loss: 0.4916 - acc: 0.7778 - val_loss: 0.4992 - val_acc: 0.7601
Epoch 98/100
2264/2264 [==============================] - 2s 718us/step - loss: 0.4940 - acc: 0.7747 - val_loss: 0.4993 - val_acc: 0.7619
Epoch 99/100
2264/2264 [==============================] - 2s 716us/step - loss: 0.4950 - acc: 0.7721 - val_loss: 0.4992 - val_acc: 0.7601
Epoch 100/100
2264/2264 [==============================] - 2s 669us/step - loss: 0.4943 - acc: 0.7787 - val_loss: 0.4993 - val_acc: 0.7601
Test accuracy: 0.7601410928436898
Train on 2264 samples, validate on 567 samples
Epoch 1/150
2264/2264 [==============================] - 3s 1ms/step - loss: 0.6485 - acc: 0.5830 - val_loss: 0.6010 - val_acc: 0.6402
Epoch 2/150
2264/2264 [==============================] - 2s 741us/step - loss: 0.5623 - acc: 

2264/2264 [==============================] - 2s 765us/step - loss: 0.4827 - acc: 0.7814 - val_loss: 0.4867 - val_acc: 0.7584
Epoch 54/150
2264/2264 [==============================] - 2s 851us/step - loss: 0.4803 - acc: 0.7845 - val_loss: 0.4866 - val_acc: 0.7566
Epoch 55/150
2264/2264 [==============================] - 2s 891us/step - loss: 0.4787 - acc: 0.7867 - val_loss: 0.4867 - val_acc: 0.7584
Epoch 56/150
2264/2264 [==============================] - 2s 854us/step - loss: 0.4805 - acc: 0.7800 - val_loss: 0.4867 - val_acc: 0.7566
Epoch 57/150
2264/2264 [==============================] - 2s 789us/step - loss: 0.4752 - acc: 0.7809 - val_loss: 0.4867 - val_acc: 0.7566
Epoch 58/150
2264/2264 [==============================] - 2s 750us/step - loss: 0.4808 - acc: 0.7809 - val_loss: 0.4868 - val_acc: 0.7584
Epoch 59/150
2264/2264 [==============================] - 2s 803us/step - loss: 0.4796 - acc: 0.7787 - val_loss: 0.4866 - val_acc: 0.7566
Epoch 60/150
2264/2264 [=======================

2264/2264 [==============================] - 2s 773us/step - loss: 0.4778 - acc: 0.7787 - val_loss: 0.4866 - val_acc: 0.7584
Epoch 113/150
2264/2264 [==============================] - 2s 952us/step - loss: 0.4759 - acc: 0.7809 - val_loss: 0.4866 - val_acc: 0.7566
Epoch 114/150
2264/2264 [==============================] - 2s 846us/step - loss: 0.4786 - acc: 0.7743 - val_loss: 0.4865 - val_acc: 0.7584
Epoch 115/150
2264/2264 [==============================] - 2s 737us/step - loss: 0.4845 - acc: 0.7827 - val_loss: 0.4865 - val_acc: 0.7584
Epoch 116/150
2264/2264 [==============================] - 2s 731us/step - loss: 0.4794 - acc: 0.7787 - val_loss: 0.4865 - val_acc: 0.7584
Epoch 117/150
2264/2264 [==============================] - 2s 792us/step - loss: 0.4793 - acc: 0.7783 - val_loss: 0.4864 - val_acc: 0.7584
Epoch 118/150
2264/2264 [==============================] - 2s 759us/step - loss: 0.4799 - acc: 0.7805 - val_loss: 0.4865 - val_acc: 0.7584
Epoch 119/150
2264/2264 [================

...

2264/2264 [==============================] - 2s 847us/step - loss: 0.5001 - acc: 0.7756 - val_loss: 0.4978 - val_acc: 0.7725
Epoch 8/200
2264/2264 [==============================] - 2s 774us/step - loss: 0.4940 - acc: 0.7708 - val_loss: 0.4938 - val_acc: 0.7584
Epoch 9/200
2264/2264 [==============================] - 2s 664us/step - loss: 0.4978 - acc: 0.7712 - val_loss: 0.4993 - val_acc: 0.7672
Epoch 10/200
2264/2264 [==============================] - 2s 677us/step - loss: 0.4940 - acc: 0.7761 - val_loss: 0.4923 - val_acc: 0.7672
Epoch 11/200
2264/2264 [==============================] - 2s 673us/step - loss: 0.4901 - acc: 0.7814 - val_loss: 0.4896 - val_acc: 0.7566
Epoch 12/200
2264/2264 [==============================] - 2s 693us/step - loss: 0.4931 - acc: 0.7805 - val_loss: 0.4912 - val_acc: 0.7672
Epoch 13/200
2264/2264 [==============================] - 1s 658us/step - loss: 0.4952 - acc: 0.7809 - val_loss: 0.4936 - val_acc: 0.7707
Epoch 14/200
2264/2264 [=========================

Epoch 65/200
2264/2264 [==============================] - 2s 840us/step - loss: 0.4844 - acc: 0.7800 - val_loss: 0.4877 - val_acc: 0.7584
Epoch 66/200
2264/2264 [==============================] - 2s 780us/step - loss: 0.4794 - acc: 0.7809 - val_loss: 0.4877 - val_acc: 0.7566
Epoch 67/200
2264/2264 [==============================] - 2s 767us/step - loss: 0.4774 - acc: 0.7831 - val_loss: 0.4878 - val_acc: 0.7584
Epoch 68/200
2264/2264 [==============================] - 2s 671us/step - loss: 0.4753 - acc: 0.7827 - val_loss: 0.4876 - val_acc: 0.7531
Epoch 69/200
2264/2264 [==============================] - 2s 685us/step - loss: 0.4777 - acc: 0.7814 - val_loss: 0.4876 - val_acc: 0.7531
Epoch 70/200
2264/2264 [==============================] - 2s 668us/step - loss: 0.4850 - acc: 0.7858 - val_loss: 0.4876 - val_acc: 0.7531
Epoch 71/200
2264/2264 [==============================] - 2s 665us/step - loss: 0.4788 - acc: 0.7822 - val_loss: 0.4876 - val_acc: 0.7531
Epoch 72/200
2264/2264 [==========

2264/2264 [==============================] - 2s 698us/step - loss: 0.4787 - acc: 0.7774 - val_loss: 0.4873 - val_acc: 0.7549
Epoch 125/200
2264/2264 [==============================] - 2s 832us/step - loss: 0.4816 - acc: 0.7814 - val_loss: 0.4872 - val_acc: 0.7584
Epoch 126/200
2264/2264 [==============================] - 2s 795us/step - loss: 0.4824 - acc: 0.7809 - val_loss: 0.4872 - val_acc: 0.7584
Epoch 127/200
2264/2264 [==============================] - 2s 789us/step - loss: 0.4780 - acc: 0.7778 - val_loss: 0.4872 - val_acc: 0.7584
Epoch 128/200
2264/2264 [==============================] - 2s 896us/step - loss: 0.4756 - acc: 0.7805 - val_loss: 0.4874 - val_acc: 0.7566
Epoch 129/200
2264/2264 [==============================] - 2s 776us/step - loss: 0.4792 - acc: 0.7836 - val_loss: 0.4872 - val_acc: 0.7584
Epoch 130/200
2264/2264 [==============================] - 2s 795us/step - loss: 0.4821 - acc: 0.7787 - val_loss: 0.4872 - val_acc: 0.7584
Epoch 131/200
2264/2264 [================

2264/2264 [==============================] - 2s 681us/step - loss: 0.4764 - acc: 0.7822 - val_loss: 0.4870 - val_acc: 0.7566
Epoch 184/200
2264/2264 [==============================] - 2s 695us/step - loss: 0.4777 - acc: 0.7836 - val_loss: 0.4869 - val_acc: 0.7549
Epoch 185/200
2264/2264 [==============================] - 2s 731us/step - loss: 0.4772 - acc: 0.7840 - val_loss: 0.4868 - val_acc: 0.7549
Epoch 186/200
2264/2264 [==============================] - 2s 748us/step - loss: 0.4780 - acc: 0.7814 - val_loss: 0.4868 - val_acc: 0.7549
Epoch 187/200
2264/2264 [==============================] - 2s 713us/step - loss: 0.4752 - acc: 0.7836 - val_loss: 0.4869 - val_acc: 0.7566
Epoch 188/200
2264/2264 [==============================] - 2s 725us/step - loss: 0.4869 - acc: 0.7849 - val_loss: 0.4868 - val_acc: 0.7531
Epoch 189/200
2264/2264 [==============================] - 2s 722us/step - loss: 0.4731 - acc: 0.7787 - val_loss: 0.4867 - val_acc: 0.7566
Epoch 190/200
2264/2264 [================

2264/2264 [==============================] - 2s 832us/step - loss: 0.4862 - acc: 0.7849 - val_loss: 0.4884 - val_acc: 0.7601
Epoch 42/50
2264/2264 [==============================] - 2s 869us/step - loss: 0.4843 - acc: 0.7818 - val_loss: 0.4884 - val_acc: 0.7601
Epoch 43/50
2264/2264 [==============================] - 2s 807us/step - loss: 0.4862 - acc: 0.7831 - val_loss: 0.4884 - val_acc: 0.7601
Epoch 44/50
2264/2264 [==============================] - 2s 812us/step - loss: 0.4813 - acc: 0.7836 - val_loss: 0.4884 - val_acc: 0.7601
Epoch 45/50
2264/2264 [==============================] - 2s 870us/step - loss: 0.4833 - acc: 0.7778 - val_loss: 0.4884 - val_acc: 0.7601
Epoch 46/50
2264/2264 [==============================] - 2s 885us/step - loss: 0.4848 - acc: 0.7805 - val_loss: 0.4883 - val_acc: 0.7601
Epoch 47/50
2264/2264 [==============================] - 2s 776us/step - loss: 0.4827 - acc: 0.7822 - val_loss: 0.4884 - val_acc: 0.7601
Epoch 48/50
2264/2264 [==============================

In [ ]:
0.86434